# Apriori Algorithm

Apriroi works by joining and pruning every items in dataset. We first define the `min_support` as the threshold for determining which items to save. Here are the steps to explain how aprori works
1. Seperate the list in each item into different items
2. Keep the item that emerge more than the threshold and delete the ones that does not
3. Join each item and list every combination
4. Repeat the steps from second step

## 1. Generating Frequent Itemsets
We're going to create a dataset that contains lists of items. This dataset is going to be used as our primary dataset to determine which the association rules between certain items

In [9]:
dataset = [['Math', 'Physics', 'Chemistry', 'Biology'],
['Math', 'Physics', 'Chemistry'], ['Math', 'Economy', 'Sociology'],
['Chemistry', 'Biology'], ['Sociology, Geography'], ['Physics', 'Chemistry'],
['Chemistry, Sociology', 'Biology'], ['Math', 'Economy', 'Physics']
]

Since the `apriori` function expects data to be in a one-hot encoded dataframe, then we can transform our dataset using TransactionEncoder

In [11]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder

ModuleNotFoundError: No module named 'mlxtend'